In [7]:
//var Collection = require('dstools').Collection;
//var data = Collection().loadCSV('data.csv');
// var data = Collection([{field1:2,field2:4},{field1:3,field2:5},{field1:2,field2:4}
//                        ,{field1:4,field2:4},{field1:4,field2:4},{field1:2,field2:4}]);

//var data = Collection().loadCSV('heart.csv');
//var lib = require('../lib')


var fs = require('fs');
var _init = fs.readFileSync('init.js').toString();
eval(_init);
setupD3();


D3 - loader


In [3]:
var ff = require('../lib');
//var Frame = ff.Frame;

undefined

In [8]:
function doN(n, fn) {
    return () =>{
        var res;
        for(let i=0;i<n; i++) { res = fn(res); }
        return res;
    }
}

undefined

In [9]:
var ICD10 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var A = r => r.ICD10[0] === 'A';
ICD10.slice(0,20);

bytes read 13187299
Lines:  150410


Ix,ICD10,TYPE,DESCRIPTION
0,A00.0,CM,"Cholera due to Vibrio cholerae 01, biovar cholerae"
1,A00.1,CM,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,CM,"Cholera, unspecified"
3,A01.00,CM,"Typhoid fever, unspecified"
4,A01.01,CM,Typhoid meningitis
5,A01.02,CM,Typhoid fever with heart involvement
6,A01.03,CM,Typhoid pneumonia
7,A01.04,CM,Typhoid arthritis
8,A01.05,CM,Typhoid osteomyelitis
9,A01.09,CM,Typhoid fever with other complications


In [31]:
 var _ = timeOf(doN(1, () => ICD10.data.filter(r => r[0][0] === 'A' )));
12000/150000*1000

Call to func took 11 milliseconds.


80

In [22]:
var CNT = 1
var myData = timeOf(doN(CNT, () =>ICD10.filter(r => r.ICD10.match(/^A/) ).project(['ICD10'])))
var __ = timeOf(doN(CNT, () =>ICD10.filter(r => r.ICD10.startsWith('A') ).project(['ICD10'])))
var ___ = timeOf(doN(CNT, () =>ICD10.filter(A).project(['ICD10'])))
var ____ = timeOf(doN(CNT, () =>ICD10.filter(A)))



Call to func took 28 milliseconds.
Call to func took 27 milliseconds.
Call to func took 15 milliseconds.
Call to func took 20 milliseconds.


undefined

In [10]:
var getMethods = (obj) => Object.getOwnPropertyNames(obj).filter(item => typeof obj[item] === 'function')

console.log('len', ICD10.length)
var x =timeOf(doN(CNT,() => ICD10.innerJoin(myData, [...ICD10.columns.map(x => '1.'+x)], 'ICD10')))//.slice(10,20)
ICD10.project(["T", 'TYPE', 'ICD10'], {T: r => r.ICD10[0] },true)
    .groupBy(['T', 'TYPE', gb.count('ICD10', 'CNT')])
    .sort(['-CNT']);
//gb

len 150409
Call to func took 332 milliseconds.


Ix,T,TYPE,CNT
0,0,PCS,68471
1,S,CM,30711
2,T,CM,9808
3,M,CM,6484
4,V,CM,3726
5,H,CM,3220
6,B,PCS,2941
7,O,CM,2250
8,D,PCS,1939
9,Y,CM,1485


In [11]:
function renameColumns(frame,map) {
	return frame.columns.map(c => map[c]?map[c]:c);

}



undefined

In [60]:
var NAMES = new Frame([['PCS', "Procedure"], ['CM', "Diagnostic"],["???", "Unknown"]], ["TYPE", "DESC"], "ICD-TYPE");
var VALUES = new Frame([['PCS', "Institution"], ['CM', "Any"],["UNK", "Unknown"]], ["TYPE", "LOC"], "ICD-TYPE");



keys(gb)

[
  'stdDiv', 'mean',
  'count',  'sum',
  'min',    'max',
  'merge',  'mergeU'
]

In [64]:
NAMES.leftJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC=TYPE'}), "TYPE")
      .groupBy(['TYPE', gb.count('TYPE','CNT')])//.groupBy([gb.min('CNT')])

Ix,TYPE,CNT
0,Diagnostic,71704
1,Procedure,78705
2,Unknown,1


In [36]:
NAMES.leftJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE')

Ix,TYPE,DESC,LOC
0,PCS,Procedure,Institution
1,CM,Diagnostic,Any
2,???,Unknown,


In [44]:
NAMES.outerJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE').sort(["DESC", "TYPE"])

Ix,TYPE,DESC,LOC
0,CM,Diagnostic,Any
1,PCS,Procedure,Institution
2,???,Unknown,
3,UNK,,Unknown


In [26]:
NAMES.innerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

Ix,ICD10,DESC,DESCRIPTION
0,A00.0,Diagnostic,"Cholera due to Vibrio cholerae 01, biovar cholerae"
1,A00.1,Diagnostic,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,Diagnostic,"Cholera, unspecified"
3,A01.00,Diagnostic,"Typhoid fever, unspecified"
4,A01.01,Diagnostic,Typhoid meningitis
5,A01.02,Diagnostic,Typhoid fever with heart involvement
6,A01.03,Diagnostic,Typhoid pneumonia
7,A01.04,Diagnostic,Typhoid arthritis
8,A01.05,Diagnostic,Typhoid osteomyelitis
9,A01.09,Diagnostic,Typhoid fever with other complications


In [27]:
NAMES.outerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

Ix,ICD10,DESC,DESCRIPTION
0,0016070,Procedure,"Bypass Cerebral Ventricle to Nasopharynx with Autologous Tissue Substitute, Open Approach"
1,0016071,Procedure,"Bypass Cerebral Ventricle to Mastoid Sinus with Autologous Tissue Substitute, Open Approach"
2,0016072,Procedure,"Bypass Cerebral Ventricle to Atrium with Autologous Tissue Substitute, Open Approach"
3,0016073,Procedure,"Bypass Cerebral Ventricle to Blood Vessel with Autologous Tissue Substitute, Open Approach"
4,0016074,Procedure,"Bypass Cerebral Ventricle to Pleural Cavity with Autologous Tissue Substitute, Open Approach"
5,0016075,Procedure,"Bypass Cerebral Ventricle to Intestine with Autologous Tissue Substitute, Open Approach"
6,0016076,Procedure,"Bypass Cerebral Ventricle to Peritoneal Cavity with Autologous Tissue Substitute, Open Approach"
7,0016077,Procedure,"Bypass Cerebral Ventricle to Urinary Tract with Autologous Tissue Substitute, Open Approach"
8,0016078,Procedure,"Bypass Cerebral Ventricle to Bone Marrow with Autologous Tissue Substitute, Open Approach"
9,001607B,Procedure,"Bypass Cerebral Ventricle to Cerebral Cisterns with Autologous Tissue Substitute, Open Approach"


In [ ]:
var {Frame} = require('../dist/bundle.js');

In [ ]:
var C = class {
    constructor(a) {
        this.value = a;
    }
    
    
};

var c = new C(1);
new c.constructor(2);
function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

In [ ]:
var la = require('../lib/array')

In [ ]:
function test(n) {
    var a = la.newArray(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test0(n) {
    var a = new Int32Array(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test2(n) {
    var a = [];
    for(let i=0; i<n; i++) a.push(i);
    return a;
}

function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

[timeOf(() => la.arrSum(test(10000000))),
timeOf(() => la.arrSum(test0(10000000)))];
var a = test0(10000000);
var b = test2(10000000);


In [ ]:
[timeOf(() => {for(i=0; i<100; i++)la.arrSum(b)}),
timeOf(() => {for(i=0; i<100; i++)la.arrSum(a)})]

In [ ]:
//var {Frame} = require('../lib/frame')

In [ ]:
var f = new Frame( [[1,2],[3,4]],['a','b'],'test');
ArrayBuffer.isView(new Int32Array(10))

In [ ]:
f.sort(['-b'])

In [ ]:
fs.readdirSync('../data/reference');
var ICD9 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var HCPCS = readZipFrame('../data/reference/hcpcs.tsv.gz')
HCPCS = HCPCS.project(['TYPE', ...HCPCS.columns], {TYPE: r => r.CODE.substr(0,2)},true);
HCPCS.groupBy(['TYPE', gb.count('TYPE', 'COUNT')]).sort(['-COUNT'])

In [ ]:
HCPCS.filter(r => r.TYPE === '90')//.groupBy(['TYPE', gb.count('TYPE', 'COUNT')])


In [ ]:
$$.html(`<script>
console.log(define.toString())
</script>`)


In [ ]:
var CFrame = class {
    constructor(data, columns, isColumn=true) {
        this.columns = columns;
        let len = isColumn? data[0].length : data.length;
        this.length = len;
        if(!column) {
           let rLn = this.columns.length;
           let aData = arrayOf(() => new Array(len), rLn);
           this.data = aData;
           for(let i=0; i<len; i++) {
               let row = data[i];
               let j = 0;
               for(; j< rLn-7; j+= 8) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
                   aData[j+4][i] = row[j+4];
                   aData[j+5][i] = row[j+5];
                   aData[j+6][i] = row[j+6];
                   aData[j+7][i] = row[j+7];
               }
               for(; j< rLn-3; j+= 4) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
               }
               for(; j< rLn; j++) {
                   aData[j][i] = row[j]
               }
           }
       }
    }

    /**
     * old Column array
    _colMapper(mapArray, baseData) {
        
    }*/
}

#Adam optimizer

In [ ]:
/*
  Adam optimizer javascript implementation based on the original article: https://arxiv.org/pdf/1412.6980.pdf
  Code written by Damir Vodenicarevic (https://damip.net) in march 2017.
  Updated by Nurul Choudhury
  This work is licensed under a Creative Commons Attribution 4.0 International License.
  http://creativecommons.org/licenses/by/4.0/
*/

function adam_optimizer(nparams, beta_1=0.9, beta_2=0.999, epsilon=1e-8) {
    var self= this;

    self.nparams= nparams;
    self.beta_1= beta_1;
    self.beta_2= beta_2;
    self.epsilon= epsilon;
    
    self.m= []; 
    self.v= [];
    for(var i= 0 ; i < self.nparams ; ++i) {
        self.m[i]= 0; // Initialize 1st moment vector
        self.v[i]= 0; // Initialize 2nd moment vector
    }
    self.t= 0; //Initialize timestep
    self.b1t = beta_1;
    self.b2t = beta_2;
    self.get_update= function(alpha, g) { // alpha is the learning rate, g is the gradient
        self.t += 1;
        let updates= new Array(self.nparams);
        let m = self.m;
        let v = self.v;
        let beta_1 = self.beta_1;
        let beta_2 = self.beta_2, b1t = self.b1t, b2t= self.b2t;
        let epsilon = self.epsilon;
        for(var i= 0 ; i < self.nparams ; ++i) {
            m[i]= beta_1*m[i] + (1-beta_1)*g[i]; // Update biased first moment estimate
            v[i]= beta_2*v[i] + (1-beta_2)*g[i]*g[i]; // Update biased second raw moment estimate
            var mub= m[i]/(1 - b1t); // Compute bias-corrected first moment estimate
            var vub= self.v[i]/(1 - b2t); // Compute bias-corrected second raw moment estimate
            updates[i]= - alpha * mub / ( Math.sqrt(vub) + epsilon  ); // Return parameter updates
           // console.log({b1t:self.b1t,b2t:self.b2t});
        }
        self.b1t *= beta_1*beta_1;
        self.b2t *= beta_2*beta_2;
        return updates;
    };
};

var Adam = class {
    constructor(nparams, beta_1=0.9, beta_2=0.999, epsilon=1e-8) {
        this.nparams= nparams;
        this.beta_1= beta_1;
        this.beta_2= beta_2;
        this.epsilon= epsilon;

        this.m= []; 
        this.v= [];
        for(var i= 0 ; i < this.nparams ; ++i) {
            this.m[i]= 0; // Initialize 1st moment vector
            this.v[i]= 0; // Initialize 2nd moment vector
        }
        this.t= 0; //Initialize timestep
        this.b1t = beta_1;
        this.b2t = beta_2;
    }
    get_update(alpha, g) { // alpha is the learning rate, g is the gradient
        this.t += 1;
        let updates= new Array(this.nparams);
        const {m,v,b,beta_1,beta_2, b1t, b2t, epsilon, nparams} = this;
         
        for(var i= 0 ; i < nparams ; ++i) {
            m[i]= beta_1*m[i] + (1-beta_1)*g[i]; // Update biased first moment estimate
            v[i]= beta_2*v[i] + (1-beta_2)*g[i]*g[i]; // Update biased second raw moment estimate
            var mub= m[i]/(1 - b1t); // Compute bias-corrected first moment estimate
            var vub= v[i]/(1 - b2t); // Compute bias-corrected second raw moment estimate
            updates[i]= - alpha * mub / ( Math.sqrt(vub) + epsilon  ); // Return parameter updates
        }
        this.b1t *= beta_1*beta_1;
        this.b2t *= beta_2*beta_2;
        return updates;
    };
}

In [ ]:
Adam

In [ ]:
var m = 0, v = 0, beta_1 = 0.9, beta_2 = 0.999;
var b1t = beta_1, b2t = beta_2;
function gx([x,y]) {
    return [2*x-100*2*x*(y-x*x), 100*2*(y-x*x)]
}

var fx = ([x,y]) => x*x+100*(y-x*x)*(y-x*x);

In [ ]:
function steepest_descent_optimizer(nparams) {
    var self= this;

    self.nparams= nparams;
    
    self.get_update= function(alpha, g) { // alpha is the learning rate, g is the gradient
        var updates= [];
        for(var i= 0 ; i < self.nparams ; ++i) {
            updates[i]= - alpha * g[i]; // Return parameter updates
        }
        //console.log('updates:',updates);
        return updates;
    };
};

function adam_optimizerOld(nparams, beta_1=0.9, beta_2=0.999, epsilon=1e-8) {
    var self= this;

    self.nparams= nparams;
    self.beta_1= beta_1;
    self.beta_2= beta_2;
    self.epsilon= epsilon;
    
    self.m= []; 
    self.v= [];
    for(var i= 0 ; i < self.nparams ; ++i) {
        self.m[i]= 0; // Initialize 1st moment vector
        self.v[i]= 0; // Initialize 2nd moment vector
    }
    self.t= 0; //Initialize timestep
    
    self.get_update= function(alpha, g) { // alpha is the learning rate, g is the gradient
        self.t += 1;
        var t = self.t;
        var updates= [];
        for(var i= 0 ; i < self.nparams ; ++i) {
            self.m[i]= self.beta_1*self.m[i] + (1-self.beta_1)*g[i]; // Update biased first moment estimate
            self.v[i]= self.beta_2*self.v[i] + (1-self.beta_2)*g[i]*g[i]; // Update biased second raw moment estimate
            var mub= self.m[i]/(1 - Math.pow(self.beta_1,2*t-1)); // Compute bias-corrected first moment estimate
            var vub= self.v[i]/(1 - Math.pow(self.beta_2,2*t-1)); // Compute bias-corrected second raw moment estimate
            updates[i]= - alpha * mub / ( Math.sqrt(vub) + self.epsilon  ); // Return parameter updates
           //console.log({b1t:Math.pow(self.beta_1,2*t-1),b2t:Math.pow(self.beta_2,2*t-1)})
        }
        return updates;
    };
};

In [ ]:
var g = 0.015;
var epsilon=1e-8;
var alpha = 0.05;
var theta = 5;

gx

In [ ]:
for(var i=0; i<100; i++){
    g = gx(theta);
    m = beta_1*m + (1-beta_1)*g;
    v = beta_2*v + (1-beta_2)*g*g;
    var mub =m/(1-b1t);
    var vub =v/(1-b2t);
    b1t = beta_1*b1t;
    b2t = beta_2*b2t;
    var update = - alpha * mub / ( Math.sqrt(vub) + epsilon  );
    theta += update;
    
}
console.log({update, theta,mub, vub: Math.sqrt(vub),m, b1t, b2t, g, g2: g*g})

In [ ]:
var theta= [1,1]; // Initialize parameters

var sphere_function= function(theta) {
    var res= 0;
    for(var i= 0 ; i < theta.length ; ++i) {
        res += theta[i]*theta[i];
    }
    return res;
}

var sphere_function_gradient= function(theta) {
    var res= [0,0];
    for(var i= 0 ; i < theta.length ; ++i) {
        res[i]= 2*theta[i];
    }
    return res;
}

/*
  Binary Descent optimizer javascript implementation
  Code written by Damir Vodenicarevic (https://damip.net) in march 2017.
  This work is licensed under a Creative Commons Attribution 4.0 International License.
  http://creativecommons.org/licenses/by/4.0/
*/

function binary_descent_optimizer(nparams) {
    var self= this;

    self.nparams= nparams;
    
    self.get_update= function(alpha, g) { // alpha is the learning rate, g is the gradient
        var updates= [];
        for(var i= 0 ; i < self.nparams ; ++i) {
            updates[i]= alpha * (g[i] >= 0 ? -1 : +1); // Return parameter updates
        }
        return updates;
    };
};

//var optimizer= new binary_descent_optimizer(theta.length); // Choose optimizer here
var optimizer= new steepest_descent_optimizer(theta.length); // Choose optimizer here

var alpha= 0.05; // Learning rate
var nsteps= 200; // Number of optimization steps
for(var step= 0; step < nsteps ; ++step) {
    //var update= optimizer.get_update(alpha, sphere_function_gradient(theta)); // get the update from the optimizer 
    var update= optimizer.get_update(alpha, gx(theta)); // get the update from the optimizer 
  for(var i= 0 ; i < theta.length ; ++i) {
    theta[i] += update[i]; //apply the update
  }
  //var value= sphere_function(theta);
  var value= fx(theta);
  
  
}
console.log('Current function value: ' + value, theta);

In [ ]:
function fit(genOptimizer,grad=gx,func=fx,_steps=190,alpha=0.05,beta_1=0.9, beta_2=0.999) {
    var theta= [1,1]; // Initialize parameters
    var optimizer= new genOptimizer(theta.length,beta_1,beta_2); // Choose optimizer here

    //alpha=  Learning rate
    var nsteps= _steps; // Number of optimization steps
    for(var step= 0; step < nsteps ; ++step) {
      var update= optimizer.get_update(alpha, grad(theta)); // get the update from the optimizer 
      for(var i= 0 ; i < theta.length ; ++i) {
        theta[i] += update[i]; //apply the update
      }
      var value= func(theta);

      //console.log('Current function value: ' + value, theta, gx(theta), update);
    }
    console.log('Current function value: ' + value, theta, grad(theta), update);
    return theta;
}

//fit(adam_optimizerOld,sphere_function_gradient,sphere_function,100);
fit(adam_optimizerOld,gx,fx,500,0.1);

//sphere_function(0.12461140440061709, 0.15483380486489354)

In [ ]:
 //fit(adam_optimizer,sphere_function_gradient,sphere_function,100);
timeOf(() =>fit(Adam,gx,fx,500,0.3));
timeOf(() =>fit(binary_descent_optimizer,sphere_function_gradient,sphere_function,300));

//fit(steepest_descent_optimizer,sphere_function_gradient,sphere_function,100);
//fit(binary_descent_optimizer,130);

In [ ]:
 function arrEQ(arr1,arr2) {
	if( arr1 === arr2) return true;
	if(arr1.length !== arr2.length) return false;

	let len = Math.min(arr1.length, arr2.length);
	let i=0;
	if( len >= 8) {
		let len1 = len - (len&7);
		for(; i<len1; i += 8) {
			if(arr1[i]   !== arr2[i])   return false;
			if(arr1[i+1] !== arr2[i+1]) return false;
			if(arr1[i+2] !== arr2[i+2]) return false;
			if(arr1[i+3] !== arr2[i+3]) return false;
			if(arr1[i+4] !== arr2[i+4]) return false;
			if(arr1[i+5] !== arr2[i+5]) return false;
			if(arr1[i+6] !== arr2[i+6]) return false;
			if(arr1[i+7] !== arr2[i+7]) return false;
		} 
	} 
	if( len >= 4 ){
		let len1 = len - (len&3);
		for(; i<len1; i += 4) {
			if(arr1[i]   !== arr2[i])   return false;
			if(arr1[i+1] !== arr2[i+1]) return false;
			if(arr1[i+2] !== arr2[i+2]) return false;
			if(arr1[i+3] !== arr2[i+3]) return false;
		}    
	}

	switch(len-i) {
	case 3: if(arr1[i+2] !== arr2[i+2]) return false;
	case 2: if(arr1[i+1] !== arr2[i+1]) return false;
	case 1: if(arr1[i] !== arr2[i]) return false;
	}

	return true;
}

function test(LEN,ix) {
    var arr1 = [0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19].slice(0,LEN);
    var arr2 = [0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19].slice(0,LEN);
    var arr3 = arr1.map( i => ix === i ?-1: i)
    console.log(LEN, arrEQ(arr1,arr2), arrEQ(arr3,arr2), arr3.join(','),"...", arr2.join(','))
}

for(var j = 0; j<50; j++) test(j, j-1)


In [ ]:
function findIndexOf(aSet, searchElement) { 
	let minIndex = 0;
	let maxIndex = aSet.length - 1;
	let currentIndex;
	let currentElement;
 	


	while (minIndex <= maxIndex) {

		currentIndex = (minIndex + maxIndex) >>> 1;
		currentElement = aSet[currentIndex];
 
		if (currentElement < searchElement) {
			minIndex = currentIndex + 1;
		}
		else if (currentElement > searchElement) {
			maxIndex = currentIndex - 1;
		}
		else {
			return currentIndex;
		}
	}
 
	return -1;
}

function range(n) {
    res = [];
    for(let i=0; i<n; i++) res.push(i);
    return range;
}

function iter(n,fn=Identity) {
    let res = [];
    for(let i=0; i<n; i++) {
        res.push(fn(i));
    }
    return res;
}
function test(LEN,ix) {
    //var arr1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40].slice(0,LEN);
    var arr1 = iter(LEN+10);
    console.log(LEN, findIndexOf(arr1,ix), findIndexOf(arr1,ix-2), findIndexOf(arr1,ix-3), findIndexOf(arr1,5))
}

timeOf(() =>{ for(var j = 0; j<41; j++) test(j, j-1) })

In [76]:
var result;
var http = require('https');
http.get("https://raw.githubusercontent.com/cmusam/fortune500/master/csv/fortune500-2017.csv",
         (resp) =>{ 
  let data = '';

  // A chunk of data has been recieved.
  resp.on('data', (chunk) => {
    data += chunk;
  });

  // The whole response has been received. Print out the result.
  resp.on('end', () => {
    console.log(data);
    result = data;
  });
})

ClientRequest {
  _events: [Object: null prototype] {
    response: [Function: bound onceWrapper] { listener: [Function] },
    socket: [Function: bound onceWrapper] { listener: [Function: onSocket] }
  },
  _eventsCount: 2,
  _maxListeners: undefined,
  outputData: [
    {
      data: 'GET /cmusam/fortune500/master/csv/fortune500-2017.csv HTTP/1.1\r\n' +
        'Host: raw.githubusercontent.com\r\n' +
        'Connection: close\r\n' +
        '\r\n',
      encoding: 'latin1',
      callback: [Function: bound onFinish]
    }
  ],
  outputSize: 118,
  writable: true,
  _last: true,
  chunkedEncoding: false,
  shouldKeepAlive: false,
  useChunkedEncodingByDefault: false,
  sendDate: false,
  _removedConnection: false,
  _removedContLen: false,
  _removedTE: false,
  _contentLength: 0,
  _hasBody: true,
  _trailer: '',
  finished: true,
  _headerSent: true,
  socket: null,
  connection: null,
  _header: 'GET /cmusam/fortune500/master/csv/fortune500-2017.csv HTTP/1.1\r\n' +
    'Host: raw.

rank,company,revenue ($ millions),profit ($ millions)
1,Walmart,485873,13643
2,Berkshire Hathaway,223604,24074
3,Apple,215639,45687
4,Exxon Mobil,205004,7840
5,McKesson,192487,2258
6,UnitedHealth Group,184840,7017
7,CVS Health,177526,5317
8,General Motors,166380,9427
9,AT&T,163786,12976
10,Ford Motor,151800,4596
11,AmerisourceBergen,146850,1427.9
12,Amazon.com,135987,2371
13,General Electric,126661,8831
14,Verizon Communications,125980,13127
15,Cardinal Health,121546,1427
16,Costco,118719,2350
17,Walgreens Boots Alliance,117351,4173
18,Kroger,115337,1975
19,Chevron,107567,-497
20,Fannie Mae,107162,12313
21,J.P. Morgan Chase,105486,24733
22,Express Scripts Holding,100288,3404.4
23,Home Depot,94595,7957
24,Boeing,94571,4895
25,Wells Fargo,94176,21938
26,Bank of America Corp.,93662,17906
27,Alphabet,90272,19478
28,Microsoft,85320,16798
29,Anthem,84863,2469.8
30,Citigroup,82386,14912
31,Comcast,80403,8695
32,IBM,79919,11872
33,State Farm Insurance Cos.,76132,350.3
34,Phillips 66,72396,1555

In [77]:
ff.frameFromBuffer(result,'fortune',ff.csvLine)

Ix,rank,company,revenue ($ millions),profit ($ millions)
0,1,Walmart,485873,13643
1,2,Berkshire Hathaway,223604,24074
2,3,Apple,215639,45687
3,4,Exxon Mobil,205004,7840
4,5,McKesson,192487,2258
5,6,UnitedHealth Group,184840,7017
6,7,CVS Health,177526,5317
7,8,General Motors,166380,9427
8,9,AT&T,163786,12976
9,10,Ford Motor,151800,4596
